# Coherent dark states and polarization switching
Studying the effect of polarization switching on coherent dark states in a 9-level system. The system is made of two ground states, one excited state but all with J = 1 for a total of nine levels. Basically just 3x the 3-level system studied in "Coherent dark states in a 3-level system.ipynb".

## Imports
Start by importing the necessary packages

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import numpy as np
import scipy
import qutip
from sympy import Symbol

from toy_systems.couplings import FirstRankCouplingJ, ToyEnergy
from toy_systems.dark_states import get_dark_states
from toy_systems.decays import CouplingDecay
from toy_systems.hamiltonian import Hamiltonian
from toy_systems.quantum_system import QuantumSystem
from toy_systems.states import Basis, BasisState, JQuantumNumbers

## Set up states and basis
We start by defining the three states of the system: the ground states $|g0\rangle$ and $|g1\rangle$, and the excited state $|e\rangle$>

In [2]:
g0s = [BasisState(qn=JQuantumNumbers(label="g0", J = 1, mJ = mJ)) for mJ in np.arange(-1,2)]
g1s = [BasisState(qn=JQuantumNumbers(label="g1", J = 1, mJ = mJ)) for mJ in np.arange(-1,2)]
es = [BasisState(qn=JQuantumNumbers(label="e", J = 1, mJ = mJ)) for mJ in np.arange(-1,2)]

# Define basis
basis = Basis(g0s+g1s+es)
basis.print()

|0> = |J = 1, mJ = -1, label = g0>
|1> = |J = 1, mJ = 0, label = g0>
|2> = |J = 1, mJ = 1, label = g0>
|3> = |J = 1, mJ = -1, label = g1>
|4> = |J = 1, mJ = 0, label = g1>
|5> = |J = 1, mJ = 1, label = g1>
|6> = |J = 1, mJ = -1, label = e>
|7> = |J = 1, mJ = 0, label = e>
|8> = |J = 1, mJ = 1, label = e>


## Define energies, couplings, decays and quantum system
I'm going to define the system in the rotating frame as usual.

### Energies

In [3]:
δ = Symbol('delta') # Energy splitting between |g0> and |g1
Δ = Symbol('Delta') # Detuning of drive field from 0
E0 = ToyEnergy(g0s, -δ/2)
E1 = ToyEnergy(g1s, +δ/2)
Ee = ToyEnergy(es, Δ)

### Couplings
Will treat the problem as if it has two time dependent laser fields: one polarized along z and the other one along x. The polarization will rotate back and forth between the two directions

In [4]:
Ω0 = Symbol('Omega_0') # Drive field Rabi rate for |g0> <-> |e>
Ω1 = Symbol('Omega_1') # Drive field Rabi rate for |g1> <-> |e>

# A condition to make sure the ground states don't get coupled to each other
def both_not_ground(state1, state2):
    return (state1.qn.label == 'e') or (state2.qn.label == 'e')

coupling_z = FirstRankCouplingJ(Ω0, p_car=np.array((0,0,1)), other_conds = [both_not_ground])
coupling_x = FirstRankCouplingJ(Ω1, p_car=np.array((1,0,0)), other_conds = [both_not_ground])

### Decays
Defining a decay from all $|e\rangle$ to all $|g0\rangle$ and $|g1\rangle$ as permitted by angular momentum:

In [5]:
# Define dipole couplings that connect excited state to ground states
decay_couplings = [
    FirstRankCouplingJ(1, p_car=np.array((1,0,0)), other_conds=[both_not_ground]),
    FirstRankCouplingJ(1, p_car=np.array((0,1,0)), other_conds=[both_not_ground]),
    FirstRankCouplingJ(1, p_car=np.array((0,0,1)), other_conds=[both_not_ground]),
]

decays = [CouplingDecay(e, Symbol("Gamma"), decay_couplings) for e in es] 

### Define a QuantumSystem
The QuantumSystem object combines the basis, Hamiltonian and decays to make setting parameters for time evolution using QuTiP more convenient.

In [6]:
# Define the system
system = QuantumSystem(
    basis=basis,
    couplings=[E0, E1, Ee, coupling_z, coupling_x],
    decays=decays,
)

# Get representations of the Hamiltonian and the decays that will be accepted by qutip
Hqobj, c_qobj = system.get_qobjs()

In [7]:
system.couplings[3].qobj

(Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
 Qobj data =
 [[ 0.          0.          0.          0.          0.          0.
   -0.40824829  0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.
    0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.
    0.          0.          0.40824829]
  [ 0.          0.          0.          0.          0.          0.
   -0.40824829  0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.
    0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.
    0.          0.          0.40824829]
  [-0.40824829  0.          0.         -0.40824829  0.          0.
   -0.40824829  0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.
    0.          0.          0.        ]
  [ 0.          0.          0.40824829  0.          